In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [0]:
def load_table_rdd(path):
    """
    Load data from a CSV file into an RDD.
    Assumes the CSV has a header which we skip.
    """
    rdd = sc.textFile(path)
    header = rdd.first() # extract header
    rdd = rdd.filter(lambda row: row != header) # filter out header
    return rdd

In [0]:
conf = SparkConf().setAppName("FCCG RDD Implementation")
sc = SparkContext.getOrCreate(conf=conf)

# Load data into an RDD
rdd = load_table_rdd('/FileStore/tables/web_Stanford.txt')
#rdd = load_table_rdd('/FileStore/tables/web_NotreDame.txt')

In [0]:
def clean_table_rdd(rdd):
    """
    Clean the RDD by splitting each line and converting to integer keys and values.
    """
    return rdd.map(lambda line: line.split('\t')) \
              .map(lambda kv: (int(kv[0]), int(kv[1])))

In [0]:
rdd = clean_table_rdd(rdd)

In [0]:
def fccg_iterate(rdd):
    # MAP
    rdd_mapped = rdd.union(rdd.map(lambda kv: (kv[1], kv[0])))
    
    # REDUCE
    # Create adjacency list and find minimum value in each list
    rdd_reduced = rdd_mapped.groupByKey().mapValues(lambda values: (list(set(values)), min(values)))
    
    # Filter the rows that have min_value lower than key
    rdd_filtered = rdd_reduced.filter(lambda kv: kv[0] > kv[1][1])
    
    # Calculate the new pairs created
    new_count = rdd_filtered.map(lambda kv: len(kv[1][0]) - 1).sum() # - rdd_filtered.count()
    
    # Prepare for next iteration
    rdd_final = rdd_filtered.flatMap(lambda kv: [(k, kv[1][1]) for k in kv[1][0] + [kv[0]] ]).distinct()
    
    return rdd_final, new_count

In [0]:
def compute_fccg(rdd):
    """Main computation loop to find connected components."""
    nb_iteration = 0
    
    while True:
        nb_iteration += 1
        
        rdd, count = fccg_iterate(rdd)

        print(f"Number of new pairs for iteration #{nb_iteration}:\t{count}")
        if count == 0:
            print("\nNo new pair, end of computation")
            break
            
    return rdd

In [0]:
final_rdd = compute_fccg(rdd)

Number of new pairs for iteration #1:	3525475
Number of new pairs for iteration #2:	1635518
Number of new pairs for iteration #3:	625639
Number of new pairs for iteration #4:	643507
Number of new pairs for iteration #5:	684584
Number of new pairs for iteration #6:	347208
Number of new pairs for iteration #7:	40104
Number of new pairs for iteration #8:	14550
Number of new pairs for iteration #9:	12241
Number of new pairs for iteration #10:	9791
Number of new pairs for iteration #11:	5558
Number of new pairs for iteration #12:	1497
Number of new pairs for iteration #13:	740


In [0]:
# Number of connected components in the graph
final_rdd.map(lambda x: x[1]).distinct().count()


Out[67]: 1